<a href="https://colab.research.google.com/github/thirza258/myKaggle/blob/main/CuGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CuGraph Submission


In [163]:
# !pip install nx-arangodb
# !pip install nx-cugraph-cu12 --extra-index-url https://pypi.nvidia.com
# !pip install --upgrade langchain langchain-community langchain-openai langgraph
# !pip install -qU faiss-cpu

In [164]:
import networkx as nx
import nx_arangodb as nxadb

import os
from google.colab import userdata

from arango import ArangoClient

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from random import randint
import re

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool

In [165]:
from arango import ArangoClient
import base64

encodedCA = "LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURHRENDQWdDZ0F3SUJBZ0lRSXlzVUd4M0hNazl4NzVXVHlmZ2E4akFOQmdrcWhraUc5dzBCQVFzRkFEQW0KTVJFd0R3WURWUVFLRXdoQmNtRnVaMjlFUWpFUk1BOEdBMVVFQXhNSVFYSmhibWR2UkVJd0hoY05NalV3TWpFNApNREl3TVRNd1doY05NekF3TWpFM01ESXdNVE13V2pBbU1SRXdEd1lEVlFRS0V3aEJjbUZ1WjI5RVFqRVJNQThHCkExVUVBeE1JUVhKaGJtZHZSRUl3Z2dFaU1BMEdDU3FHU0liM0RRRUJBUVVBQTRJQkR3QXdnZ0VLQW9JQkFRQ2QKOGNDcElVQTNkaHY3T0oyblFzNDZoVTZJQkpxWnk0R0ZrS1d4SXN5eWlxWGpvSVpDWFZobTV2WFNkdG1WWHBXSgpwMHVUWm1mWkt2eHcwYmFPbGM1MGJ3ZG9jOS9ZdWlXbzI3QTEzVUllS0JuSFVySVhtd3hpNm9OaFBOZG5KNHBnCkZYMXJvN2hsOHdXWENzZ1owNFV4dlJTa2FSZDliZktpeXVXVW1wU3ZkSFZJVGxMUHJQTXJjMWhGcWFmRE00STkKTGVNYkwvNkRra1hla3RyKzZVYVJSVnM1K3pqMFhLZ0xUZlNzek0zRkkxcktXS3RVczdCNlpmRzk4aHJsSW1sOQp6MWx0bTEyZk5OeDFvUkFsbEI3a0M0bFJhdmQ3NmlOTFdianBGZm4rZmxPOEpZakI2RDZ4NW95S3NNN0pWRjAxClhxYUVOU1Mrdkp0QnFaYlZNKy8xQWdNQkFBR2pRakJBTUE0R0ExVWREd0VCL3dRRUF3SUNwREFQQmdOVkhSTUIKQWY4RUJUQURBUUgvTUIwR0ExVWREZ1FXQkJSK29PYlBlMDM3VjZXRzI0b1dUbUFhb2VpQzFEQU5CZ2txaGtpRwo5dzBCQVFzRkFBT0NBUUVBaTBIREhHQjFLTXdhdUdSV2NRTUlyYjN4OFJCM083cmtLeXFnNUd3MTVoNUNEMUUyCkVaa3V4LzdPWjg1QW1jb3JCZlc2ekZGQlVtaktITjhJZWdCUGtmOXowWHJmNzYyeHFRVHRJYlZlb1Frb0hlL2kKKzd1UzY4OUhRVjNKTVM4VWYvVE5ZUjhQRWtieUdJdzBZWEFyUUIwT2I2UzVSNklOVk1kdlM3a2MrMVJyODBoZwpTT1Nsc25jcnRtQ1Jqd3J1MEFacG9DRy8xMXlWUlU1ZStsSjJNeURMNWtVS083bEFNZ0QvbXlOa2RHUFdUS1BsClk5L0FDS1JCbTRQeWZxYUVtcTVPbTNKVEtBS2xMckR2WWcxUVpDN0FPdkJ0L3BhMWNoQlRPaEloVmpsdW85c1MKUUM1YWRLcHdwcVkxVitPeis1Zlh0bUxuZDdlYW5TcVdLMER2YlE9PQotLS0tLUVORCBDRVJUSUZJQ0FURS0tLS0tCg=="
try:
    file_content = base64.b64decode(encodedCA)
    with open("cert_file.crt", "w+") as f:
        f.write(file_content.decode("utf-8"))
except Exception as e:
    print(str(e))
    exit(1)

client = ArangoClient(
    hosts="https://b3889dc81c5a.arangodb.cloud:18529", verify_override="cert_file.crt"
)

db = client.db("_system", username="root", password=userdata.get("DB_PASSWORD"))

# Note that ArangoGraph Insights Platform runs deployments in a cluster configuration.
# To achieve the best possible availability, your client application has to handle
# connection failures by retrying operations if needed.
print("ArangoDB:", db.version())

ArangoDB: 3.12.4


# Data Preprocessing

# The list of all articles.
# Article names are URL-encoded; e.g., in Java they can be decoded using java.net.URLDecoder.decode(articleName, "UTF-8").
# FORMAT:   article
#
# When publishing on this data set, please cite:
# (1) Robert West and Jure Leskovec:
#     Human Wayfinding in Information Networks.
#     21st International World Wide Web Conference (WWW), 2012.
# (2) Robert West, Joelle Pineau, and Doina Precup:
#     Wikispeedia: An Online Game for Inferring Semantic Distances between Concepts.
#     21st International Joint Conference on Artificial Intelligence (IJCAI), 2009.

In [166]:
articles = pd.read_csv("/content/drive/MyDrive/wikispeedia_paths-and-graph/wikispeedia_paths-and-graph/articles.tsv", sep='\t', names=["article"])

articles.head()

,article
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in
1,%C3%85land
2,%C3%89douard_Manet
3,%C3%89ire
4,%C3%93engus_I_of_the_Picts


# Hierarchical categories of all articles.
# Many articles have more than one category. Some articles have no category.
# Article names are URL-encoded; e.g., in Java they can be decoded using java.net.URLDecoder.decode(articleName, "UTF-8").
# FORMAT:   article   category
#
# When publishing on this data set, please cite:
# (1) Robert West and Jure Leskovec:
#     Human Wayfinding in Information Networks.
#     21st International World Wide Web Conference (WWW), 2012.
# (2) Robert West, Joelle Pineau, and Doina Precup:
#     Wikispeedia: An Online Game for Inferring Semantic Distances between Concepts.
#     21st International Joint Conference on Artificial Intelligence (IJCAI), 2009.

In [167]:
categories = pd.read_csv("/content/drive/MyDrive/wikispeedia_paths-and-graph/wikispeedia_paths-and-graph/categories.tsv", sep='\t', names=['article', 'category'])

categories.sample(5)

,article,category
3212,Morphine,subject.Science.Biology.Health_and_medicine
1926,Giant_Panda,subject.Science.Biology.Mammals
1795,Frederick_Hamilton-Temple-Blackwood%2C_1st_Mar...,subject.People.Political_People
3999,Royal_Marines,subject.History.Military_History_and_War
2510,James_Lovelock,subject.People.Writers_and_critics


# The list of all links between articles.
# Article names are URL-encoded; e.g., in Java they can be decoded using java.net.URLDecoder.decode(articleName, "UTF-8").
# FORMAT:   linkSource   linkTarget
#
# When publishing on this data set, please cite:
# (1) Robert West and Jure Leskovec:
#     Human Wayfinding in Information Networks.
#     21st International World Wide Web Conference (WWW), 2012.
# (2) Robert West, Joelle Pineau, and Doina Precup:
#     Wikispeedia: An Online Game for Inferring Semantic Distances between Concepts.
#     21st International Joint Conference on Artificial Intelligence (IJCAI), 2009.

In [168]:
links = pd.read_csv("/content/drive/MyDrive/wikispeedia_paths-and-graph/wikispeedia_paths-and-graph/links.tsv", sep='\t', names=['linkSource', 'linkTarget'])

links.head()

,linkSource,linkTarget
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Bede
1,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Columba
2,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,D%C3%A1l_Riata
3,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Great_Britain
4,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Ireland


# Successful (i.e., finished) Wikispeedia paths.
# Article names are URL-encoded; e.g., in Java they can be decoded using java.net.URLDecoder.decode(articleName, "UTF-8").
# Articles in a path are separated by ";".
# Back clicks are represented as "<".
# Ratings are optionally given by the user after finishing the game and range from 1 ("easy") to 5 ("brutal").
# Missing ratings are represented as "NULL".
# FORMAT:   hashedIpAddress   timestamp   durationInSec   path   rating
#
# When publishing on this data set, please cite:
# (1) Robert West and Jure Leskovec:
#     Human Wayfinding in Information Networks.
#     21st International World Wide Web Conference (WWW), 2012.
# (2) Robert West, Joelle Pineau, and Doina Precup:
#     Wikispeedia: An Online Game for Inferring Semantic Distances between Concepts.
#     21st International Joint Conference on Artificial Intelligence (IJCAI), 2009.


In [169]:
paths_finished = pd.read_csv("/content/drive/MyDrive/wikispeedia_paths-and-graph/wikispeedia_paths-and-graph/paths_finished.tsv", sep='\t', names=["hashedIpAddress", "timestamp", "durationInSec", "path", "rating"])

paths_finished.head()

,hashedIpAddress,timestamp,durationInSec,path,rating
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0


# Unsuccessful (i.e., unfinished) Wikispeedia paths.
# Article names are URL-encoded; e.g., in Java they can be decoded using java.net.URLDecoder.decode(articleName, "UTF-8").
# Articles in a path are separated by ";".
# Back clicks are represented as "<".
# There are two types of quitting:
# (1) "timeout" means that no click was made for 30 minutes;
# (2) "restart" means that the user started a new game without finishing the current one.
# FORMAT:   hashedIpAddress   timestamp   durationInSec   path   target   type
#
# When publishing on this data set, please cite:
# (1) Robert West and Jure Leskovec:
#     Human Wayfinding in Information Networks.
#     21st International World Wide Web Conference (WWW), 2012.
# (2) Robert West, Joelle Pineau, and Doina Precup:
#     Wikispeedia: An Online Game for Inferring Semantic Distances between Concepts.
#     21st International Joint Conference on Artificial Intelligence (IJCAI), 2009.

In [170]:
paths_unfinished = pd.read_csv("/content/drive/MyDrive/wikispeedia_paths-and-graph/wikispeedia_paths-and-graph/paths_unfinished.tsv", sep='\t', names=["hashedIpAddress", "timestamp", "durationInSec", "path", "target", "type"])

paths_unfinished.head()

,hashedIpAddress,timestamp,durationInSec,path,target,type
0,2426091a53125110,1297054935,1804,Obi-Wan_Kenobi,Microsoft,timeout
1,26141fd878806294,1297055651,1805,Julius_Caesar,Caracas,timeout
2,2b015fb8181c48f2,1297090819,1818,Malawi;Democracy;Alexander_the_Great,First_Crusade,timeout
3,53a53bc244e08a6a,1297094761,49,Paraguay,Mount_St._Helens,restart
4,53a53bc244e08a6a,1297099105,1808,Paraguay;Bolivia,Mount_St._Helens,timeout


Joining The table

In [187]:
links = pd.merge(links, categories, left_on="linkSource", right_on="article", how="outer")

links.sample(5)

,linkSource,linkTarget,article_x,category_x,article_y,category_y
85239,Integrated_circuit,History,Integrated_circuit,subject.Science.Physics.Electricity_and_Electr...,Integrated_circuit,subject.IT.Computing_hardware_and_infrastructure
159590,Stephen_Hawking,China,Stephen_Hawking,subject.People.Astronomers_and_physicists,Stephen_Hawking,subject.People.Astronomers_and_physicists
59484,Fatty_acid,Palm_oil,Fatty_acid,subject.Everyday_life.Food_and_agriculture,Fatty_acid,subject.Science.Biology.Health_and_medicine
100969,Liechtenstein,Malaysia,Liechtenstein,subject.Geography.European_Geography.European_...,Liechtenstein,subject.Geography.European_Geography.European_...
83697,Idit_Harel_Caperton,China,Idit_Harel_Caperton,subject.People.Human_Scientists,Idit_Harel_Caperton,subject.People.Human_Scientists


Converting to graph


In [172]:
G = nx.from_pandas_edgelist(links, 'linkSource', 'linkTarget')

print(G)

Graph with 4593 nodes and 106648 edges


In [173]:
print(G.degree("Microsoft"))

54


In [174]:
# plot_options = {"node_size": 10, "with_labels": False, "width": 0.15}
# pos = nx.spring_layout(G, iterations=15, seed=1721)
# fig, ax = plt.subplots(figsize=(15, 9))
# nx.draw_networkx(G, pos=pos, ax=ax, **plot_options)

In [175]:
print(list(G.nodes())[:10])  # Check first 10 node IDs


['%C3%81ed%C3%A1n_mac_Gabr%C3%A1in', 'Bede', 'Columba', 'D%C3%A1l_Riata', 'Great_Britain', 'Ireland', 'Isle_of_Man', 'Monarchy', 'Orkney', 'Picts']


In [176]:
# Create the graph with overwrite enabled
G_adb = nxadb.MultiGraph(
    name="WikipediaLinks",
    db=db,
    incoming_graph_data=G,
    write_batch_size=50000,
    overwrite_graph=True
)

print(G_adb)


[10:48:57 +0000] [INFO]: Overwriting graph 'WikipediaLinks'
INFO:nx_arangodb:Overwriting graph 'WikipediaLinks'
[10:48:57 +0000] [INFO]: Graph 'WikipediaLinks' exists.
INFO:nx_arangodb:Graph 'WikipediaLinks' exists.
[10:48:57 +0000] [INFO]: Default node type set to 'WikipediaLinks_node'
INFO:nx_arangodb:Default node type set to 'WikipediaLinks_node'
[2025/03/08 10:48:58 +0000] [1484] [INFO] - adbnx_adapter: Instantiated ADBNX_Adapter with database '_system'
INFO:adbnx_adapter:Instantiated ADBNX_Adapter with database '_system'


Output()

Output()

[2025/03/08 10:49:02 +0000] [1484] [INFO] - adbnx_adapter: Created ArangoDB 'WikipediaLinks' Graph
INFO:adbnx_adapter:Created ArangoDB 'WikipediaLinks' Graph


MultiGraph named 'WikipediaLinks' with 4593 nodes and 0 edges


In [ ]:
import urllib.parse

def get_node_article_mapping(G):
    """
    Create a mapping between node IDs and article titles

    Parameters:
    G (networkx.Graph): The graph with node IDs

    Returns:
    dict: Mapping of node IDs to article titles
    """
    node_to_article = {}

    # Assuming you have a way to access the original data
    # This could be from your ArangoDB or from the original dataset
    for node_id in G.nodes():
        # Get node data from graph if it exists
        node_data = G.nodes[node_id]

        # If the node has an 'article' or 'title' attribute
        if 'article' in node_data:
            article_name = node_data['article']
            node_to_article[node_id] = urllib.parse.unquote(article_name)
        elif 'title' in node_data:
            article_name = node_data['title']
            node_to_article[node_id] = urllib.parse.unquote(article_name)

    return node_to_article

In [177]:
print(len(G_adb.nodes))  # Should print the number of nodes
print(list(G_adb.nodes())[:10])  # Print some node names


4593
['WikipediaLinks_node/1333', 'WikipediaLinks_node/1283', 'WikipediaLinks_node/3160', 'WikipediaLinks_node/2330', 'WikipediaLinks_node/140', 'WikipediaLinks_node/3024', 'WikipediaLinks_node/470', 'WikipediaLinks_node/2000', 'WikipediaLinks_node/3198', 'WikipediaLinks_node/1046']


In [178]:
G_adb = nxadb.Graph(name="WikipediaLinks", db=db)

print(G_adb)

[10:49:06 +0000] [INFO]: Graph 'WikipediaLinks' exists.
INFO:nx_arangodb:Graph 'WikipediaLinks' exists.
[10:49:07 +0000] [INFO]: Default node type set to 'WikipediaLinks_node'
INFO:nx_arangodb:Default node type set to 'WikipediaLinks_node'


Graph named 'WikipediaLinks' with 4593 nodes and 6647 edges


In [179]:
arango_graph = ArangoGraph(db)

In [180]:
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

llm.invoke("hello!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'stop', 'logprobs': None}, id='run-fd7108e9-4311-430d-b9ae-a5e0cbdb202d-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [181]:
@tool
def text_to_aql_to_text(query: str):
  """This tool is available to invoke the
    ArangoGraphQAChain object, which enables you to
    translate a Natural Language Query into AQL, execute
    the query, and translate the result back into Natural Language.
    """
  llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

  chain = ArangoGraphQAChain.from_llm(
    	llm=llm,
    	graph=arango_graph,
    	verbose=True,
        allow_dangerous_requests=True
    )
  result = chain.invoke(query)

  return str(result["result"])


In [182]:
@tool
def recommend_articles_from_graph(query):
    """This tool recommends Wikipedia articles based on a user query
    and the link structure of the ArangoDB Graph.

    - It interprets the query to determine the most relevant articles.
    - Uses NetworkX algorithms (PageRank, Personalized PageRank, etc.).
    - Returns a list of recommended articles.
    """

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    ######################
    print("1) Identifying Relevant Algorithm")

    text_to_nx = llm.invoke(f"""
    I have a directed NetworkX Graph called {G_adb} representing Wikipedia articles and their hyperlinks.

    **Graph Schema:**
    - **Nodes:** Wikipedia article titles.
    - **Edges:** Hyperlinks between articles (A → B means article A links to article B).
    - {arango_graph.schema}

    **User Query:** {query}

    **Task:** Identify the best algorithm to generate recommendations based on:
    - Similar articles based on link structure.
    - PageRank-based importance.
    - Articles frequently linked together.

    **Expected Output:**
    Return only the name of the best algorithm (e.g., "Personalized PageRank", "HITS", "Common Neighbors").
    Do not include any extra text.
    """).content.strip()

    print(f"Selected Algorithm: {text_to_nx}")

    FINAL_RESULT = get_recommendations(G=G_adb, method=text_to_nx, query_article=query, num_results=10)

    return FINAL_RESULT


In [183]:
def get_recommendations(G, method, query_article=None, num_results=10):
    """
    Generate article recommendations based on various graph algorithms.

    Parameters:
    G (networkx.Graph): The graph representing article relationships
    method (str): The recommendation algorithm to use
    query_article (str, optional): The source article for personalized recommendations
    num_results (int): Number of recommendations to return

    Returns:
    list: Recommended articles sorted by relevance
    """
    recommended_articles = []

    if method == "Personalized PageRank":
        if query_article and query_article in G:
            personalization = {query_article: 1.0}
            pr = nx.pagerank(G, personalization=personalization)
            # Remove the query article from recommendations
            if query_article in pr:
                pr.pop(query_article)
            recommended_articles = sorted(pr, key=pr.get, reverse=True)[:num_results]
        else:
            # Fallback to standard PageRank if query article is invalid
            pr = nx.pagerank(G)
            recommended_articles = sorted(pr, key=pr.get, reverse=True)[:num_results]

    elif method == "HITS":
        hubs, authorities = nx.hits(G)
        recommended_articles = sorted(authorities, key=authorities.get, reverse=True)[:num_results]

    elif method == "Common Neighbors":
        if query_article and query_article in G:
            neighbors = set(G.neighbors(query_article))
            candidates = {
                node: len(neighbors & set(G.neighbors(node)))
                for node in G.nodes if node != query_article
            }
            # Sort by number of common neighbors, and only include nodes with at least one common neighbor
            filtered_candidates = {node: count for node, count in candidates.items() if count > 0}
            if filtered_candidates:
                recommended_articles = sorted(filtered_candidates, key=filtered_candidates.get, reverse=True)[:num_results]
            else:
                # Fallback if no common neighbors found
                recommended_articles = list(G.neighbors(query_article))[:num_results]
        else:
            # Fallback to degree centrality if query article is invalid
            dc = nx.degree_centrality(G)
            recommended_articles = sorted(dc, key=dc.get, reverse=True)[:num_results]
    else:
        # Default to degree centrality if method not recognized
        dc = nx.degree_centrality(G)
        recommended_articles = sorted(dc, key=dc.get, reverse=True)[:num_results]

    return recommended_articles

Getting the nodes based on input

In [184]:
recommended_articles = recommend_articles_from_graph("Ireland and Great Britain")
recommended_articles

1) Identifying Relevant Algorithm
Selected Algorithm: Personalized PageRank


[10:49:19 +0000] [INFO]: Graph 'WikipediaLinks' load took 5.162464380264282s
INFO:nx_arangodb:Graph 'WikipediaLinks' load took 5.162464380264282s
[10:49:19 +0000] [INFO]: NXCG Graph construction took 0.0034592151641845703s
INFO:nx_arangodb:NXCG Graph construction took 0.0034592151641845703s


['WikipediaLinks_node/102',
 'WikipediaLinks_node/30',
 'WikipediaLinks_node/183',
 'WikipediaLinks_node/38',
 'WikipediaLinks_node/61',
 'WikipediaLinks_node/31',
 'WikipediaLinks_node/40',
 'WikipediaLinks_node/1044',
 'WikipediaLinks_node/370',
 'WikipediaLinks_node/466']

In [194]:
# Create a mapping between node numbers and article names
def create_node_to_article_mapping(links_df):
    """
    Create a mapping between node ID numbers and article names

    Parameters:
    links_df: DataFrame containing the original link data with linkSource and linkTarget

    Returns:
    dict: Mapping of node numbers to article names
    """

    all_articles = set(links_df['linkSource'].unique()) | set(links_df['linkTarget'].unique())

    article_list = sorted(list(map(str, all_articles)))
    node_to_article = {i: article for i, article in enumerate(article_list)}

    return node_to_article

def display_readable_recommendations(recommended_ids, mapping):
    """
    Convert node IDs to article titles and display recommendations

    Parameters:
    recommended_ids: List of node IDs (e.g., ['WikipediaLinks_node/102', ...])
    mapping: Dictionary mapping node numbers to article names

    Returns:
    list: Recommendations with human-readable article titles
    """
    results = []

    for node_id in recommended_ids:
        # Extract the numeric part from the node ID
        numeric_id = int(node_id.split('/')[-1])

        if numeric_id in mapping:
            article_title = mapping[numeric_id]
            results.append({
                'id': node_id,
                'article': article_title
            })
        else:
            results.append({
                'id': node_id,
                'article': f"Unknown article (ID: {numeric_id})"
            })

    return results

node_to_article = create_node_to_article_mapping(links)

recommended_ids = recommended_articles
readable_recs = display_readable_recommendations(recommended_ids, node_to_article)

for i, rec in enumerate(readable_recs, 1):
    print(f"{i}. {rec['article']} ({rec['id']})")

1. Abuja (WikipediaLinks_node/102)
2. 2003_Atlantic_hurricane_season (WikipediaLinks_node/30)


In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import openai

# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Load text from a file and split into chunks
def load_text_chunks(file_path, chunk_size=256):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    # Split text into chunks (by sentences or fixed size)
    sentences = text.split(". ")  # Split by sentence
    chunks = [" ".join(sentences[i : i + chunk_size]) for i in range(0, len(sentences), chunk_size)]

    return chunks

# Path to your text file
text_file = "knowledge_base.txt"  # Replace with your actual file path
documents = load_text_chunks(text_file)

# Generate embeddings for each chunk
doc_embeddings = embedding_model.encode(documents)

# Create FAISS index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(doc_embeddings, dtype=np.float32))  # Add embeddings to FAISS

# Function to retrieve relevant text chunks
def retrieve(query, top_k=2):
    query_embedding = embedding_model.encode([query])
    _, indices = index.search(np.array(query_embedding, dtype=np.float32), top_k)
    return [documents[i] for i in indices[0]]

# Function to generate response using GPT-4
def generate_response(query):
    retrieved_docs = retrieve(query, top_k=2)
    context = "\n".join(retrieved_docs)

    prompt = f"Context:\n{context}\n\nUser Query: {query}\nAnswer:"

    openai.api_key = "your_openai_api_key"  # Replace with your API key
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}],
    )

    return response["choices"][0]["message"]["content"]

# Example Query
query = "What is the Eiffel Tower?"
print("User:", query)
print("RAG Response:", generate_response(query))


In [186]:
# tools = [text_to_aql_to_text, text_to_nx_algorithm_to_text]

# def query_graph(query):
#     llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
#     app = create_react_agent(llm, tools)
#     final_state = app.invoke({"messages": [{"role": "user", "content": query}]})
#     return final_state["messages"][-1].content